<a href="https://colab.research.google.com/github/mrdbourke/pytorch-deep-learning/blob/main/extras/exercises/05_pytorch_going_modular_exercise_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 05. PyTorch Going Modular Exercises

Welcome to the 05. PyTorch Going Modular exercise template notebook.

There are several questions in this notebook and it's your goal to answer them by writing Python and PyTorch code.

> **Note:** There may be more than one solution to each of the exercises, don't worry too much about the *exact* right answer. Try to write some code that works first and then improve it if you can.

## Resources and solutions

* These exercises/solutions are based on [section 05. PyTorch Going Modular](https://www.learnpytorch.io/05_pytorch_going_modular/) of the Learn PyTorch for Deep Learning course by Zero to Mastery.

**Solutions:** 

Try to complete the code below *before* looking at these.

* See a live [walkthrough of the solutions (errors and all) on YouTube](https://youtu.be/ijgFhMK3pp4).
* See an example [solutions notebook for these exercises on GitHub](https://github.com/mrdbourke/pytorch-deep-learning/blob/main/extras/solutions/05_pytorch_going_modular_exercise_solutions.ipynb).

## 1. Turn the code to get the data (from section 1. Get Data) into a Python script, such as `get_data.py`.

* When you run the script using `python get_data.py` it should check if the data already exists and skip downloading if it does.
* If the data download is successful, you should be able to access the `pizza_steak_sushi` images from the `data` directory.

In [1]:
%%writefile going_modular/get_data.py
"""
Download the data of pizza, steak, and sushi
"""
import os
import requests
import zipfile
from pathlib import Path

def download_data(data_path=Path("data/")):
    """
    Download the pizza, steak, and sushi images

    Args:
        data_path (Path): Path to download the data
    """

    # Setup path to data folder
    image_path = data_path / "pizza_steak_sushi"

    # If the image folder doesn't exist, download it and prepare it... 
    if image_path.is_dir():
        print(f"{image_path} directory exists.")
    else:
        print(f"Did not find {image_path} directory, creating one...")
        image_path.mkdir(parents=True, exist_ok=True)

    # Download pizza, steak, sushi data
    with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
        request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
        print("Downloading pizza, steak, sushi data...")
        f.write(request.content)

    # Unzip pizza, steak, sushi data
    with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
        print("Unzipping pizza, steak, sushi data...") 
        zip_ref.extractall(image_path)

    # Remove zip file
    os.remove(data_path / "pizza_steak_sushi.zip")

Writing going_modular/get_data.py


In [4]:
# Example running of get_data.py
!python ./going_modular/get_data.py

In [5]:
from going_modular import get_data

get_data.download_data()

Did not find data\pizza_steak_sushi directory, creating one...
Unzipping pizza, steak, sushi data...


## 2. Use [Python's `argparse` module](https://docs.python.org/3/library/argparse.html) to be able to send the `train.py` custom hyperparameter values for training procedures.
* Add an argument flag for using a different:
  * Training/testing directory
  * Learning rate
  * Batch size
  * Number of epochs to train for
  * Number of hidden units in the TinyVGG model
    * Keep the default values for each of the above arguments as what they already are (as in notebook 05).
* For example, you should be able to run something similar to the following line to train a TinyVGG model with a learning rate of 0.003 and a batch size of 64 for 20 epochs: `python train.py --learning_rate 0.003 batch_size 64 num_epochs 20`.
* **Note:** Since `train.py` leverages the other scripts we created in section 05, such as, `model_builder.py`, `utils.py` and `engine.py`, you'll have to make sure they're available to use too. You can find these in the [`going_modular` folder on the course GitHub](https://github.com/mrdbourke/pytorch-deep-learning/tree/main/going_modular/going_modular). 

In [20]:
%%writefile going_modular/train.py
"""
Trains a PyTorch image classification model using device-agnostic code.
"""
import os
import torch
import data_setup, engine, model_builder, utils

from torchvision import transforms

import argparse

# Set parser
parser = argparse.ArgumentParser(description="train the pytorch model")

# Set arguemnts
parser.add_argument('--train_dir')
parser.add_argument('--test_dir')
parser.add_argument('--learning_rate')
parser.add_argument('--batch_size')
parser.add_argument('--num_epochs')
parser.add_argument('--hidden_units')

args = parser.parse_args()

# Setup hyperparameters
NUM_EPOCHS = int(args.num_epochs) if args.num_epochs else  5
BATCH_SIZE = int(args.batch_size) if args.batch_size else  32
HIDDEN_UNITS = int(args.hidden_units) if args.hidden_units else  10
LEARNING_RATE = float(args.learning_rate) if args.learning_rate else  0.001

# Setup directories
train_dir = args.train_dir if args.train_dir else "data/pizza_steak_sushi/train"
test_dir = args.test_dir if args.test_dir else  "data/pizza_steak_sushi/test"

# Setup target device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Create transforms
data_transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor()
])

# Create DataLoaders with help from data_setup.py
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir, test_dir, data_transform, BATCH_SIZE)

# Create model with help from model_builder.py
model = model_builder.TinyVGG(
    3,
    HIDDEN_UNITS,
    len(class_names)
).to(device)

print(model)

# Set loss and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), LEARNING_RATE)

# Start training with help from engine.py
engine.train(model, train_dataloader, test_dataloader, optimizer, loss_fn, NUM_EPOCHS, device)

# Save the model with help from utils.py
utils.save_model(model, "models", "05(1)_going_modular_script_mode_tinyvgg_model.pth")

Overwriting going_modular/train.py


In [21]:
# Example running of train.py
!python ./going_modular/train.py --num_epochs 5 --batch_size 128 --hidden_units 128 --learning_rate 0.0003

TinyVGG(
  (conv_block_1): Sequential(
    (0): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block_2): Sequential(
    (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=21632, out_features=3, bias=True)
  )
)
Epoch: 1 | train_loss: 1.0994 | train_acc: 0.3406 | test_loss: 1.0946 | test_acc: 0.3333 | 
Epoch: 2 | train_loss: 1.0856 | train_acc: 0.3523 | test_loss: 1.0749 | test_acc: 0.3733 | 
Epoch: 3 | train_loss: 1.0592 | train_acc: 0.5176 | test_loss: 1.0514 | test_acc: 0.4000 | 
Epoch: 4 | train_loss: 1.0067 | tra


100%|██████████| 5/5 [00:08<00:00,  1.63s/it]


In [27]:
from going_modular import model_builder
import torch

loaded_model = model_builder.TinyVGG(3, 128, 3)

loaded_model.load_state_dict(torch.load(f="models/05(1)_going_modular_script_mode_tinyvgg_model.pth"))

<All keys matched successfully>

## 3. Create a Python script to predict (such as `predict.py`) on a target image given a file path with a saved model.

* For example, you should be able to run the command `python predict.py some_image.jpeg` and have a trained PyTorch model predict on the image and return its prediction.
* To see example prediction code, check out the [predicting on a custom image section in notebook 04](https://www.learnpytorch.io/04_pytorch_custom_datasets/#113-putting-custom-image-prediction-together-building-a-function). 
* You may also have to write code to load in a trained model.

In [37]:
%%writefile going_modular/predict.py
"""
Contains function to predict give iamge and model
"""
from torchvision.io import read_image
from torchvision import transforms
import torch

import argparse

import model_builder

def predict_image():
    """
    Classify the image with trained model
    """
    # Set parser
    parser = argparse.ArgumentParser(description="predict label with input image")

    # Set arguemnts
    parser.add_argument('--image')

    args = parser.parse_args()

    image_path = args.image

    device = "cuda" if torch.cuda.is_available() else "cpu"
    
    transform = transforms.Compose([
        transforms.Resize((64, 64)),
    ])

    # load image with dtype float32
    img = read_image(image_path).type(torch.float32)

    # scale from 0 to 1
    img = img / 255
    
    # resize to (64, 64)
    img = transform(img)

    # send image to target device
    img.to(device)

    # get instance of model
    loaded_model = model_builder.TinyVGG(3, 128, 3)

    # get trained parameters
    loaded_model.load_state_dict(torch.load(f="models/05(1)_going_modular_script_mode_tinyvgg_model.pth"))
    
    # predict label
    loaded_model.eval()
    
    with torch.inference_mode():
        img = img.unsqueeze(dim=0)

        pred = loaded_model(img)

    pred_label = torch.softmax(pred, dim=1).argmax(dim=1)
    
    print(['pizza', 'steak', 'sushi'][pred_label])
    return ['pizza', 'steak', 'sushi'][pred_label]

if __name__=="__main__":
    predict_image()

Overwriting going_modular/predict.py


In [38]:
# Example running of predict.py 
!python ./going_modular/predict.py --image data/pizza_steak_sushi/test/sushi/175783.jpg

sushi


c:\Users\CSR\anaconda3\envs\ml\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
